In [1]:
%load_ext jupyter_black

## Note
I was unable to create an environment that supports both tensorflow-gpu and cudf to a dependency conflict with protobuff

# nvidia cuda


``` bash
nvidia-smi --query-gpu=timestamp,name,pci.bus_id,driver_version,pstate,pcie.link.gen.max,pcie.link.gen.current,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv
nvidia-smi --query-gpu=name,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv -l 5

nvidia-smi -q -g 0 -d UTILIZATION -l
==============NVSMI LOG==============

Timestamp                                 : Sat Aug 13 18:07:02 2022
Driver Version                            : 516.59
CUDA Version                              : 11.7

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Utilization
        Gpu                               : 1 %
        Memory                            : 1 %
        Encoder                           : 0 %
        Decoder                           : 0 %
    GPU Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 6 %
        Min                               : 0 %
        Avg                               : 0 %
    Memory Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 1 %
        Min                               : 0 %
        Avg                               : 0 %
    ENC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
    DEC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
```

In [2]:
import distributed

from dask_cuda import LocalCUDACluster


def close():
    """
    for convenience if a cluster/client is already running
    this should close them to prevent starting multiple clusters
    on the same port.
    """
    try:
        client.close(), cluster.close()
    except:
        ...


close()
cluster = LocalCUDACluster()
client = distributed.Client(cluster)

client

/opt/conda/envs/rapids/lib/python3.9/site-packages/dask_cuda/utils.py:244: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
2022-09-10 22:41:57,538 - distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/griblib/notebooks/probsevere/dask-worker-space/worker-dxqr02o7', purging
2022-09-10 22:41:57,541 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 7.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35233,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 7.68 GiB
Comm: tcp://127.0.0.1:34403,Total threads: 1
Dashboard: http://127.0.0.1:36045/status,Memory: 7.68 GiB
Nanny: tcp://127.0.0.1:45775,


In [3]:
import os
from pathlib import Path

import pandas as pd
import dask.dataframe as dd
import cudf

# creating a Path object from the absolute path to our data folder
ps_data = Path.cwd().parents[1] / "data-bucket"
all_files = tuple(ps_data.rglob("*.pq"))
ps_data

PosixPath('/workspaces/griblib/data-bucket')

In [4]:
len(all_files)

2908

In [5]:
%%time
# reading with pure pandas and pyarrow
pd.concat((pd.read_parquet(file, engine="pyarrow") for file in all_files))

CPU times: user 24.9 s, sys: 6.8 s, total: 31.7 s
Wall time: 25.7 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
3,2066,1657,-1,42.099998,284,39.099998,0.29,1.94,7,0.26,...,5.374000,26,111651,2022-04-16 08:30:35,-89.620003,34.200001,-89.550003,34.270000,-89.584999,34.230000
3,1078,285,-38,37.900002,123,23.600000,0.03,0.88,0,0.00,...,-0.975000,3,113455,2022-04-16 14:24:40,-84.000000,32.040001,-83.779999,32.230000,-83.925682,32.150002
3,678,251,-160,48.299999,87,18.500000,0.06,0.95,1,0.06,...,5.381000,6,112748,2022-04-16 11:46:37,-85.440002,31.100000,-85.139999,31.280001,-85.230415,31.195000
3,27,0,0,8.400000,212,33.099998,0.00,0.39,0,0.00,...,2.418000,0,112144,2022-04-16 09:38:38,-82.930000,36.689999,-82.779999,36.790001,-82.855003,36.735001
3,206,0,0,21.500000,343,37.099998,0.00,0.65,1,0.06,...,-2.198000,0,110971,2022-04-16 05:52:39,-86.059998,35.150002,-85.720001,35.509998,-85.894997,35.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,1621,128,0,28.400000,326,43.599998,0.00,0.53,1,0.05,...,-3.702000,1,133548,2022-04-20 18:50:38,-93.040001,33.900002,-92.949997,34.000000,-92.995003,33.955002
54,974,5,-999,23.100000,317,44.500000,0.00,0.67,0,0.00,...,4.509000,1,132613,2022-04-20 14:54:36,-92.989998,35.040001,-92.919998,35.119999,-92.955002,35.080002
54,1778,1114,-205,54.900002,39,27.600000,0.04,0.37,0,0.00,...,47.654999,9,132363,2022-04-20 14:04:39,-97.870003,27.690001,-97.809998,27.740000,-97.839996,27.715000


In [6]:
%%time
# much faster to read into a dask dataframe
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf

CPU times: user 315 ms, sys: 46.9 ms, total: 362 ms
Wall time: 327 ms


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,MAXLLAZ,P98LLAZ,P98MLAZ,WETBULB_0C_HGT,PWAT,CAPE_M10M30,LJA,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
npartitions=2908,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,uint32,int32,float32,uint32,float32,float32,float32,uint32,float32,float32,float32,float32,float32,float32,uint32,float32,uint32,float32,float32,uint8,uint32,datetime64[ns],float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
%%time
# again, but the compute time to convert it back to a pandas native object is more expensive
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf.compute()

CPU times: user 23.5 s, sys: 2.09 s, total: 25.6 s
Wall time: 44.9 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
0,1524,1270,0,51.400002,36,17.900000,0.92,2.08,36,1.25,...,6.203,88,136524,2022-03-01 00:00:29,-79.209999,30.120001,-79.070000,30.280001,-79.139999,30.200001
0,61,74,0,11.100000,225,46.000000,0.00,0.45,0,0.00,...,2.775,0,137200,2022-03-01 10:42:34,-122.099998,45.380001,-121.949997,45.509998,-122.023331,45.450001
0,74,76,0,12.200000,227,45.599998,0.00,0.45,0,0.00,...,-0.083,0,137200,2022-03-01 10:40:35,-122.169998,45.330002,-121.949997,45.500000,-122.055496,45.410000
0,990,642,0,32.299999,27,13.300000,0.10,0.70,0,0.00,...,-0.477,2,137196,2022-03-01 10:38:37,-78.800003,26.600000,-78.709999,26.709999,-78.756248,26.655001
0,990,642,0,32.299999,27,13.300000,0.11,0.72,0,0.00,...,38.986,2,137196,2022-03-01 10:36:32,-78.800003,26.600000,-78.709999,26.709999,-78.756248,26.655001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1,0,-999,0.300000,27,14.900000,0.07,0.55,0,0.00,...,2.012,0,141207,2022-04-22 12:12:39,-79.430000,36.480000,-79.330002,36.570000,-79.407501,36.525002
84,0,0,-999,0.100000,35,14.400000,0.00,0.56,0,0.00,...,-3.252,0,141208,2022-04-22 12:12:39,-79.339996,36.400002,-79.220001,36.500000,-79.264999,36.455002
85,154,79,-1,10.500000,25,16.299999,0.00,0.46,0,0.00,...,-7.438,0,141209,2022-04-22 12:12:39,-119.209999,35.669998,-119.139999,35.750000,-119.175003,35.709999
86,3,0,-999,1.900000,82,8.500000,0.00,0.95,0,0.00,...,23.327,0,141210,2022-04-22 12:12:39,-78.790001,34.849998,-78.730003,34.930000,-78.754372,34.904999


In [8]:
%%time
# the parquet files we're create using dask.dataframe
gdf: cudf.DataFrame = cudf.read_parquet(all_files)
df = gdf.to_pandas()
del gdf  # delete the dataframe from gpu memory after it has been loaded to gpu ram
df

CPU times: user 8.01 s, sys: 8.98 s, total: 17 s
Wall time: 28 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
3,2066,1657,-1,42.099998,284,39.099998,0.29,1.94,7,0.26,...,5.374000,26,111651,2022-04-16 08:30:35,-89.620003,34.200001,-89.550003,34.270000,-89.584999,34.230000
3,1078,285,-38,37.900002,123,23.600000,0.03,0.88,0,0.00,...,-0.975000,3,113455,2022-04-16 14:24:40,-84.000000,32.040001,-83.779999,32.230000,-83.925682,32.150002
3,678,251,-160,48.299999,87,18.500000,0.06,0.95,1,0.06,...,5.381000,6,112748,2022-04-16 11:46:37,-85.440002,31.100000,-85.139999,31.280001,-85.230415,31.195000
3,27,0,0,8.400000,212,33.099998,0.00,0.39,0,0.00,...,2.418000,0,112144,2022-04-16 09:38:38,-82.930000,36.689999,-82.779999,36.790001,-82.855003,36.735001
3,206,0,0,21.500000,343,37.099998,0.00,0.65,1,0.06,...,-2.198000,0,110971,2022-04-16 05:52:39,-86.059998,35.150002,-85.720001,35.509998,-85.894997,35.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,1621,128,0,28.400000,326,43.599998,0.00,0.53,1,0.05,...,-3.702000,1,133548,2022-04-20 18:50:38,-93.040001,33.900002,-92.949997,34.000000,-92.995003,33.955002
54,974,5,-999,23.100000,317,44.500000,0.00,0.67,0,0.00,...,4.509000,1,132613,2022-04-20 14:54:36,-92.989998,35.040001,-92.919998,35.119999,-92.955002,35.080002
54,1778,1114,-205,54.900002,39,27.600000,0.04,0.37,0,0.00,...,47.654999,9,132363,2022-04-20 14:04:39,-97.870003,27.690001,-97.809998,27.740000,-97.839996,27.715000


In [9]:
%%time
from pathlib import Path
from contextlib import contextmanager
from dataclasses import dataclass, field
from typing import Callable, Iterable, Generator, Literal, Mapping
import pandas as pd

try:
    from cudf import DataFrame, read_parquet
    import cupy as cp
except ModuleNotFoundError:
    from cudf import DataFrame, read_parquet
    import numpy as cp

import numpy as np
import nvector as nv
from numpy.typing import NDArray


class CUDArray(NDArray):
    def get(self) -> np.ndarray:
        ...


class CudaDataFrame(pd.DataFrame):
    def to_pandas(self) -> pd.DataFrame:
        ...

    @property
    def loc(self) -> Mapping[any, "CudaDataFrame"]:
        ...

    @property
    def values(self) -> CUDArray:
        ...


class CaseInsensiveSTR(str):
    def __eq__(self, __other: str):
        return self.casefold() == __other.casefold()


@contextmanager
def parquet_session(
    files: Iterable[Path], *, engine: str = "cudf", index: list[str] = ["VALIDTIME", "ID"]
) -> Generator[CudaDataFrame, None, None]:
    """

    context manager to remove the dataframe from the gpu memory
    """
    try:
        gdf = read_parquet(files, engine=engine).set_index(index).sort_index()
        yield gdf
    finally:
        del gdf


def enforce_literal(exc: Exception = Exception, case_insensitive: bool = True):
    """
    decorator function to inforce Litteral type hint annotations on keyword arguments
    """

    def __generate(_func_kwargs: dict[str, any]):
        for k, v in _func_kwargs.items():
            if v.__class__.__name__ == "_LiteralGenericAlias":
                yield k, tuple(
                    _v if case_insensitive and not isinstance(_v, str) else CaseInsensiveSTR(_v) for _v in v.__args__
                )

    def wraps(func: Callable):
        enforce = tuple(__generate(func.__annotations__))

        def inner(*args, **kwargs):
            for k, v in enforce:
                if not kwargs[k] in v:
                    raise exc("Literal Type Enforcement")
            return func(*args, **kwargs)

        return inner

    return wraps


@dataclass(frozen=True)
class BoundingBox:
    """
    Nominatim API
    Nominatim API returns a boundingbox property of the form:

    south Latitude, north Latitude, west Longitude, east Longitude
    For example, Greater London in JSON format:

    "boundingbox":["51.2867602","51.6918741","-0.5103751","0.3340155"]


    ```
    bbox = BoundingBox(20.005, 54.995, -129.995, -60.005, shape=(7000, 2500))
    with gpu_session(files) as gdf:
        gdf["X"] = bbox.fit_array(gdf["X"].values, axis=0)
        gdf["Y"] = bbox.fit_array(gdf["Y"].values, axis=1)
    ```
    """

    south: float
    north: float
    west: float
    east: float
    shape: tuple[int, int]
    frame_e: nv.FrameE = field(repr=False, default=nv.FrameE(name="WGS84"))

    @property
    def latitudes(self) -> CUDArray:
        _, y_num = self.shape
        return cp.linspace(
            self.south,  # south lat
            self.north,  # north lat
            y_num,  # points in between
            dtype=cp.float32,
        )

    @property
    def longitudes(self) -> CUDArray:
        x_num, _ = self.shape
        return cp.linspace(
            self.west,  # west lon
            self.east,  # east lon
            x_num,  # points in between
            dtype=cp.float32,
        )

    @property
    def matrix(self) -> CUDArray:
        return cp.array(cp.meshgrid(self.latitudes, self.longitudes), dtype=cp.float32).T

    @property
    def latrange(self) -> tuple[float, float]:
        return self.north, self.south

    @property
    def lonrange(self) -> tuple[float, float]:
        return self.west, self.east

    @property
    def y_dist(self) -> CUDArray:

        return self.__calculate_step(
            self.longitudes,
            self.latrange,
            len(self.latitudes),
        )

    @property
    def x_dist(self) -> CUDArray:

        return self.__calculate_step(
            self.latitudes,
            self.lonrange,
            len(self.longitudes),
        )

    def __calculate_step(self, target: CUDArray, step_range: tuple[float, float], count: int) -> CUDArray:
        """
        calculates the geometric distance within the bounding box

        ```
        array_west, array_east = np.array(list([bbox.lonrange]) * len(bbox.lattitudes)).T

        ecef_west = bbox.frame_e.GeoPoint(bbox.lattitudes.get(), array_west, degrees=True).to_ecef_vector()

        ecef_east = bbox.frame_e.GeoPoint(bbox.lattitudes.get(), array_east, degrees=True).to_ecef_vector()

        ecef_delta = ecef_east - ecef_west

        return cp.array(ecef_delta.length) / len(bbox.longitudes)
        ```
        """

        array_a, array_b = np.array([[step_range]] * len(target)).T

        ecef_a = self.frame_e.GeoPoint(target.get(), array_a, degrees=True).to_ecef_vector()

        ecef_b = self.frame_e.GeoPoint(target.get(), array_b, degrees=True).to_ecef_vector()

        ecef_delta = ecef_a - ecef_b

        return cp.array(ecef_delta.length) / count

    def delta_a2b(self, array_a: CUDArray, array_b: CUDArray, axis: Literal["X", "Y"]) -> CUDArray:
        zeros = np.zeros(len(array_a))
        if axis == "X":
            lat_a, lat_b = array_a.get(), array_b.get()
            lon_a, lon_b = zeros, zeros
        elif axis == "Y":
            lat_a, lat_b = zeros, zeros
            lon_a, lon_b = array_a.get(), array_b.get()
        else:
            raise Exception

        ecef_a = self.frame_e.GeoPoint(lat_a, lon_a, degrees=True).to_ecef_vector()

        ecef_b = self.frame_e.GeoPoint(lat_b, lon_b, degrees=True).to_ecef_vector()

        return cp.array((ecef_a - ecef_b).length)

    @enforce_literal(KeyError)
    def fit_array(self, values: CUDArray, axis: Literal["X", "Y"]) -> tuple[CUDArray, CUDArray]:
        """
        determines the nearest grid line in a given axis for an array
        calculates the distance along that axis to the nearest line
        """
        if axis not in ("X", "Y"):
            raise Exception

        target = self.longitudes if axis == "X" else self.latitudes

        min_diff = self.diffmin(target, values[:, cp.newaxis])

        near_line = target[min_diff]

        next_line = target[min_diff + 1]

        return near_line, self.delta_a2b(near_line, next_line, axis=axis)

    def diffmin(self, target: CUDArray, values: CUDArray) -> CUDArray:
        """
        determines the nearest grid line in a given axis for an array
        calculates the distance along that axis to the nearest line
        """
        return cp.argmin(abs(target - values), axis=1)

    @property
    def extent(self):
        return [self.west, self.east, self.south, self.north]


if __name__ == "__main__":
    south, north, west, east = 20.005, 54.995, -129.995, -60.005
    shape = (7000, 2500)
    bbox = BoundingBox(south, west, north, east, shape=shape)
    a = cp.array([-90])

    bbox.fit_array(a, axis="Y")

CPU times: user 451 ms, sys: 58 ms, total: 509 ms
Wall time: 701 ms


In [10]:
@enforce_literal(FileNotFoundError)
def my_func(*, val: Literal["A", "B"] = ...):
    ...


try:
    my_func(val="C")
except FileNotFoundError:
    print("Literaly enforced")

Literaly enforced


In [11]:
%%time
PS_THRESH = 5
s, n, w, e = 24.30, 40, -103.17, -74.96
shape = np.array([6500, 3250]) // 4
se_conus = BoundingBox(s, n, w, e, shape=tuple(shape))
import textwrap


def echo(tripq: str):
    return print(textwrap.dedent(tripq))


def open_and_filter() -> pd.DataFrame:
    with parquet_session(all_files[: len(all_files) // 2]) as gdf:
        echo(
            f"""\
            PRE MASK:
            total values: {gdf.size}
            total storms: {len(gdf.index.get_level_values('ID').unique())}
        """
        )
        # bounds validation, going to shrink the dataset into a smaller one
        x, y = (gdf[dim] for dim in ("X", "Y"))
        x_mask = (x > se_conus.west) & (x < se_conus.east)
        y_mask = (y > se_conus.south) & (y < se_conus.north)
        # grouping by the STORM ID and to call the aggregate method to get thw
        # max value for a given storm
        ps = gdf["PS"].groupby(level="ID").agg("max")
        # masking the index where the ID is in the met condition
        ps_mask = gdf.index.get_level_values("ID").isin(ps.index[ps > PS_THRESH])
        # index the dataframe with the the met conditions
        gdf = gdf.loc[x_mask & y_mask & ps_mask]
        # XXX POST MASK XXX
        echo(
            f"""\
            POST MASK:
            total values: {gdf.size}
            total storms: {len(gdf.index.get_level_values('ID').unique())}
            """
        )
        # XXX FITTING INTO GRID XXX
        echo(
            f"""\
            PRE FIT:
            unique x: {len(gdf['X'].unique())}
            unique y: {len(gdf['Y'].unique())}
            """
        )
        x, step_x = se_conus.fit_array(gdf["X"].values, axis="X")
        y, step_y = se_conus.fit_array(gdf["Y"].values, axis="Y")
        gdf["X"] = x
        gdf["STEPX"] = step_x
        gdf["Y"] = y
        gdf["STEPY"] = step_y
        del x, step_x, y, step_y

        echo(
            f"""\
            POST FIT:
            unique x: {len(gdf['X'].unique())}
            unique y: {len(gdf['Y'].unique())}
            """
        )
        return gdf.to_pandas()


df = open_and_filter()

PRE MASK:
total values: 19670418
total storms: 153053

POST MASK:
total values: 6503544
total storms: 32652

PRE FIT:
unique x: 66116
unique y: 3139

POST FIT:
unique x: 1620
unique y: 812

CPU times: user 1.88 s, sys: 3.03 s, total: 4.91 s
Wall time: 3.13 s


In [12]:
df

MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
VALIDTIME           ID                                                  
2022-03-01 00:00:29 136524    1524    1270      0  51.400002       36   
                    136658     612     267    -27  41.000000       65   
2022-03-01 00:02:37 136524    1522    1268      0  51.000000       36   
                    136658     602     260    -22  41.000000       63   
                    136681    1880    1584   -999  44.599998       36   
...                            ...     ...    ...        ...      ...   
2022-04-22 23:58:42 143049    2690    2282    -20  58.200001      156   
                    143157    2581    2404    -17  59.200001      166   
                    143209    2814    2623    -20  58.799999      151   
                    143226    2209    2035    -22  51.799999      153   
                    143333    2862    2290    -21  67.500000      116   

                            MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
VALIDTIME           ID                                                         
2022-03-01 00:00:29 136524          17.900000  0.92         2.08          36   
                    136658           3.200000  0.08         1.32           0   
2022-03-01 00:02:37 136524          18.000000  0.90         1.76          36   
                    136658           3.600000  0.08         1.81           0   
                    136681          18.000000  0.19         0.43           5   
...                                       ...   ...          ...         ...   
2022-04-22 23:58:42 143049          52.700001  0.56         1.32           4   
                    143157          49.599998  0.36         1.13           1   
                    143209          47.700001  1.17         2.55          24   
                    143226          53.000000  1.88         3.38         134   
                    143333          42.500000  1.35         2.96          23   

                            FLASH_DENSITY  ...  MOTION_SOUTH   PS        MINX  \
VALIDTIME           ID                     ...                                  
2022-03-01 00:00:29 136524           1.25  ...      6.203000   88  -79.209999   
                    136658           0.00  ...      6.624000    3  -80.489998   
2022-03-01 00:02:37 136524           1.25  ...      6.061000   86  -79.190002   
                    136658           0.00  ...      6.427000    3  -80.529999   
                    136681           0.43  ...      4.771000   11  -78.190002   
...                                   ...  ...           ...  ...         ...   
2022-04-22 23:58:42 143049           0.14  ...    -18.007000   52 -101.470001   
                    143157           0.12  ...    -16.405001   23 -101.870003   
                    143209           1.00  ...    -17.403999   97 -101.830002   
                    143226           0.78  ...     -1.182000  100 -101.440002   
                    143333           0.63  ...    -19.145000   98 -102.260002   

                                 MINY        MAXX       MAXY           X  \
VALIDTIME           ID                                                     
2022-03-01 00:00:29 136524  30.120001  -79.070000  30.280001  -79.146339   
                    136658  30.340000  -80.430000  30.400000  -80.466507   
2022-03-01 00:02:37 136524  30.129999  -79.089996  30.250000  -79.146339   
                    136658  30.320000  -80.410004  30.459999  -80.466507   
                    136681  29.730000  -78.129997  29.799999  -78.156204   
...                               ...         ...        ...         ...   
2022-04-22 23:58:42 143049  37.779999 -101.269997  37.919998 -101.363449   
                    143157  36.180000 -101.669998  36.310001 -101.762978   
                    143209  35.730000 -101.540001  35.939999 -101.745605   
                    143226  38.320000 -101.150002  38.770000 -101.259224   
                    143333  34.250000 -102.070000  34.400002 -102.16249

In [13]:
%%time


def localize(files: Iterable[Path], bbox: BoundingBox, ps_thresh: int = 0) -> pd.DataFrame:

    with parquet_session(files) as gdf:
        # bounds validation, going to shrink the dataset into a smaller one
        x, y = (gdf[dim] for dim in ("X", "Y"))
        x_mask = (x > bbox.west) & (x < bbox.east)
        y_mask = (y > bbox.south) & (y < bbox.north)
        # grouping by the STORM ID and to call the aggregate method to get thw
        # max value for a given storm
        ps = gdf["PS"].groupby(level="ID").agg("max")
        # masking the index where the ID is in the met condition
        ps_mask = gdf.index.get_level_values("ID").isin(ps.index[ps > ps_thresh])
        # index the dataframe with the the met conditions
        gdf = gdf.loc[x_mask & y_mask & ps_mask]
        x, step_x = bbox.fit_array(gdf["X"].values, axis="X")
        y, step_y = bbox.fit_array(gdf["Y"].values, axis="Y")
        gdf["X"] = x
        gdf["STEPX"] = step_x
        gdf["Y"] = y
        gdf["STEPY"] = step_y
        del x, y, step_x, step_y

        return gdf.to_pandas()


def main():
    s, n, w, e = 25.0, 30.0, -90.0, -75.0
    shape = np.array([7000, 2500]) // 4
    bbox = BoundingBox(s, n, w, e, shape=tuple(shape))
    return localize(all_files[: len(all_files) // 2], bbox)


if __name__ == "__main__":
    df = main()
df[["STEPX", "STEPY"]]

CPU times: user 1.44 s, sys: 1.54 s, total: 2.98 s
Wall time: 1.71 s


STEPX       STEPY
VALIDTIME           ID                            
2022-03-01 00:00:29 136648  957.883598  892.006079
2022-03-01 00:02:37 136648  957.188439  892.006076
                    136681  957.808253  891.773993
2022-03-01 00:04:33 136648  957.188439  892.006047
                    136681  957.808253  891.773993
...                                ...         ...
2022-04-22 21:18:40 142929  957.095747  892.000760
2022-04-22 21:54:38 143017  957.381108  891.773993
2022-04-22 21:56:35 143017  957.381108  891.773993
2022-04-22 22:08:41 143059  957.108748  892.148224
2022-04-22 22:26:41 143108  957.918410  892.153849

[102966 rows x 2 columns]

In [14]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
PS_THRESH = 40
COUNT = 60

idx: slice = pd.IndexSlice
s, n, w, e = 25.0, 35.0, -95.0, -80.0
shape = np.array([700, 250]) // 4
bbox = BoundingBox(s, n, w, e, shape=tuple(shape))
df = localize(all_files[:len(all_files)//2], bbox, ps_thresh=PS_THRESH)

ax:GeoAxes = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(np.array(bbox.extent) * 1.005)
ax.coastlines(resolution="50m")
ax.gridlines()

mask = df.index.get_level_values("ID").value_counts() > COUNT

data = df.loc[idx[:, mask[mask].index], :].unstack("ID")
# data["color"] = "black"
ax.plot("X","Y", color="k", linewidth=.5, data=data)




: 

: 

In [ ]:
from datetime import timedelta, datetime
mask = abs(df.index.get_level_values("VALIDTIME") - datetime.fromisoformat("2022-04-21")) > timedelta(days=2)
df[mask] 

In [ ]:
a = [-5, -4, -2, 0, 2, 4, 5]
num = 2
delta = [abs(i - num) for i in a]
gt = 3
mask = [i < gt for i in delta]
[val for cond, val in zip(mask, a) if cond]
from typing import TypeVar, Iterable, Callable

VT = TypeVar("VT", datetime, int, float)


def mask_with_callback(a: Iterable[VT], func: Callable[[VT], bool]) -> list[VT]:
    return [i for i in a if func(i)]


_2023_01_01 = datetime.fromisoformat("2023-01-01T00:00:00")
_30_days = timedelta(days=30)
_now = datetime.utcnow()
date_range = [_now + timedelta(days=i) for i in range(365)]

out = mask_with_callback(
    date_range,
    lambda date: abs(date - _2023_01_01) < _30_days,
)
out

In [ ]:
%%time
def assert_bbox(south:float, north:float, west:float, east:float,*,shape:tuple[7000,2500] )->BoundingBox:
    bbox = BoundingBox(south, north, west, east, shape=shape)
    assert (
        (bbox.latitudes.min() == south)
        & (bbox.latitudes.max() == north)
        & (bbox.longitudes.min() == west)
        & (bbox.longitudes.max() == east)
    )
    assert shape == (*bbox.y_dist.shape, *bbox.x_dist.shape)
    assert cp.array(cp.meshgrid(bbox.x_dist, bbox.y_dist)).T.shape == bbox.matrix.shape
    return bbox

def fit2grid(values:CUDArray,target:CUDArray, )->CUDArray:
    return target[cp.argmin(abs(target - values[:, np.newaxis]), axis=1)]

